<a href="https://colab.research.google.com/github/manjotmb20/tweet-analysis/blob/master/FinalModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
data=pd.read_csv("tweets.csv")
data.dropna(inplace=True,axis=0)
data.drop(["Unnamed: 0"],axis=1,inplace=True)
!pip install tweet-preprocessor
import preprocessor as p

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7947 sha256=97f9c2779cbdd756c0b59ec03a717d6d6093129c545771c9333351401c01221f
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor


In [0]:
f, ax = plt.subplots(figsize=(10, 8))
corr = data.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax,annot=True)

In [0]:
list2=[]
for i in data.lang.unique():
  if len(data[data.lang==i])>100 and i!="und":
    list2.append(i)


In [15]:
!pip install emoji --upgrade
import emoji

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=ec6732b31c3bed0399ff6787a1fa57f5e3e07c69599fe781d0aae0ad72aabdaf
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
def print_score(y_val,y_pred):
  print("accuracy score :", accuracy_score(y_val,y_pred))
  print("confusion matrix:",confusion_matrix(y_val,y_pred))

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier

In [0]:
def preprocess_tweet(tweet):
    #converting all to lower case
    tweet=tweet.lower()
    #converting all tweet urls to string "URL"
    tweet=re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #converting all @username to "AT_USER"
    tweet=re.sub('@[^\s]+','AT_USER',tweet)
    #removing numbers from tweets
    tweet= re.sub(r'\d+','', tweet)
    #converting all multiple white spaces to a single space
    tweet=re.sub('[\s]+',' ',tweet)
    #converting hastags to name after hastags
    tweet=re.sub(r'#([^\s]+)',r'Hashtag',tweet)
    #converting emojis to words
    tweet=emoji.demojize(tweet)
    tweet = re.sub(r'[?|$|.|!|ÿ|œ¹|ð]','',tweet)
    #Removing special characters such as punctuation from tweets
    tweet=tweet.replace(r'[^a-zA-Z_]','')
    #Stemming words in text
    word=' '.join(([word for word in tweet.split()]))
    return word

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000,ngram_range=(1,1))

In [0]:
dt_clf_en=DecisionTreeClassifier()
dt_clf_ar=DecisionTreeClassifier()
dt_clf_es=DecisionTreeClassifier()
dt_clf_pt=DecisionTreeClassifier()
dt_clf_in=DecisionTreeClassifier()
dt_clf_fr=DecisionTreeClassifier()
dtclf_dict=({'ar':dt_clf_ar,'en':dt_clf_en,'es':dt_clf_es,'fr':dt_clf_fr,'pt':dt_clf_pt,'in':dt_clf_in})

In [0]:
lr_clf_en=LogisticRegression()
lr_clf_ar=LogisticRegression()
lr_clf_es=LogisticRegression()
lr_clf_pt=LogisticRegression()
lr_clf_in=LogisticRegression()
lr_clf_fr=LogisticRegression()
lrclf_dict=({'ar':lr_clf_ar,'en':lr_clf_en,'es':lr_clf_es,'fr':lr_clf_fr,'pt':lr_clf_pt,'in':lr_clf_in})

In [0]:
tfidf_en = TfidfVectorizer(max_features=1000,ngram_range=(1,1))
tfidf_ar = TfidfVectorizer(max_features=1000,ngram_range=(1,1))
tfidf_es = TfidfVectorizer(max_features=1000,ngram_range=(1,1))
tfidf_pt = TfidfVectorizer(max_features=1000,ngram_range=(1,1))
tfidf_in = TfidfVectorizer(max_features=1000,ngram_range=(1,1))
tfidf_fr = TfidfVectorizer(max_features=1000,ngram_range=(1,1))
tfidf_dict=({'ar':tfidf_ar,'en':tfidf_en,'es':tfidf_es,'fr':tfidf_fr,'pt':tfidf_pt,'in':tfidf_in})

In [93]:
lang_dict=({'en':'english','es':'spanish','fr':'french','pt':'portuguese','in':'indonesian','ar':'arabic'})
for lang in list2:
  df_eng=pd.DataFrame()
  df_eng=data[data.lang==lang]
  from nltk.corpus import stopwords
  stop = stopwords.words(lang_dict[lang])
  from nltk.stem import PorterStemmer
  final_list=[]
  labels=df_eng.label
  for row in df_eng["tweet_text"]:
    final_list.append((preprocess_tweet(row)))
  df_new=pd.DataFrame({'clean_text':final_list,'label':labels})
  x_train,x_val,y_train,y_val=train_test_split(df_new['clean_text'],df_new['label'],test_size=0.2,random_state=9)
  xtrain_tfidf=tfidf_dict[lang].fit_transform(x_train)
  xval_tfidf=tfidf_dict[lang].transform(x_val)
  print("Shape of train data for lang {} is {}".format(lang_dict[lang],xtrain_tfidf.shape))
  print("Shape of test data for lang {} is {}".format(lang_dict[lang],xval_tfidf.shape))
  print("Logistic Regression Model")
  lrclf_dict[lang].fit(xtrain_tfidf,y_train)
  y_pred=lrclf_dict[lang].predict(xval_tfidf)
  print_score(y_val,y_pred)
  print("Decision Tree Classifier")
  dtclf_dict[lang].fit(xtrain_tfidf,y_train)
  y_pred=dtclf_dict[lang].predict(xval_tfidf)
  print_score(y_val,y_pred)


Shape of train data for lang english is (5881, 1000)
Shape of test data for lang english is (1471, 1000)
Logistic Regression Model
accuracy score: 0.7824609109449354
confusion matrix: [[435 180]
 [140 716]]
Decision Tree Classifier


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy score: 0.7273963290278722
confusion matrix: [[435 180]
 [221 635]]
Shape of train data for lang french is (97, 656)
Shape of test data for lang french is (25, 656)
Logistic Regression Model
accuracy score: 0.72
confusion matrix: [[17  0]
 [ 7  1]]
Decision Tree Classifier
accuracy score: 0.84
confusion matrix: [[16  1]
 [ 3  5]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Shape of train data for lang spanish is (489, 1000)
Shape of test data for lang spanish is (123, 1000)
Logistic Regression Model
accuracy score: 0.8048780487804879
confusion matrix: [[99  0]
 [24  0]]
Decision Tree Classifier
accuracy score: 0.7967479674796748
confusion matrix: [[87 12]
 [13 11]]
Shape of train data for lang portuguese is (269, 1000)
Shape of test data for lang portuguese is (68, 1000)
Logistic Regression Model
accuracy score: 0.7794117647058824
confusion matrix: [[52  2]
 [13  1]]
Decision Tree Classifier
accuracy score: 0.7647058823529411
confusion matrix: [[46  8]
 [ 8  6]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Shape of train data for lang indonesian is (110, 960)
Shape of test data for lang indonesian is (28, 960)
Logistic Regression Model
accuracy score: 0.8571428571428571
confusion matrix: [[24  0]
 [ 4  0]]
Decision Tree Classifier
accuracy score: 0.9285714285714286
confusion matrix: [[24  0]
 [ 2  2]]
Shape of train data for lang arabic is (143, 1000)
Shape of test data for lang arabic is (36, 1000)
Logistic Regression Model
accuracy score: 0.7777777777777778
confusion matrix: [[28  0]
 [ 8  0]]
Decision Tree Classifier
accuracy score: 0.7777777777777778
confusion matrix: [[25  3]
 [ 5  3]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [105]:
test_list=[]
test="Picking up Togo order from wings n ale they have cleaned this AT_USER AT_USER #place up, looks good , https://ehfhefbest.com  https://ehfhefbest.com wings n lex "
test_list.append(preprocess_tweet(test))
tf_idf=tfidf_dict['en'].transform(test_list)
lrclf_dict['en'].predict(tf_idf)

array(['Genuine'], dtype=object)